# KeywordGacha_NV_v0.5.0

- **作者**: [https://github.com/neavo/KeywordGacha](https://github.com/neavo/KeywordGacha)

## 使用方法

* 导入models:
从 `fatecyx/my_model` 中导入 `llama-box` 和 `glm-4-9b-chat-q4_k_s` 模型。

* 导入datasets
导入 `fatecyx/KeywordGacha2` 数据集。

* 导入输入数据
根据实际目录修改命令中的`/kaggle/input/gt-input/gt_input`:

```bash
!cp -r /kaggle/input/gt-input/gt_input /kaggle/working/KeywordGacha/input
```

In [ ]:
!cp -r /kaggle/input/my_model/gguf/llama-box/1/llama-box /kaggle/working/llama-box
!chmod a+x /kaggle/working/llama-box
!cp -r /kaggle/input/keywordgacha2/KeywordGacha /kaggle/working/KeywordGacha
!cp -r /kaggle/input/gt-input/gt_input /kaggle/working/KeywordGacha/input
%cd /kaggle/working/KeywordGacha
!pip install -q -r requirements.txt

In [ ]:
import subprocess
import threading
import time

def local_model(dic_status):
    cmd = "/kaggle/working/llama-box -c 8192 -np 4 -ngl 999 --host 0.0.0.0 -m /kaggle/input/my_model/gguf/glm-4-9b-chat-q4_k_s/1/glm-4-9b-chat-Q4_K_S.gguf"
    
    
    p = subprocess.Popen(cmd, shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.PIPE)
    for line in p.stderr:
        if not dic_status['status']:
            s2 = line.decode()
            dic_status['msg'] = s2
            if "main: starting server" in s2:
                dic_status['status'] = True
            if "main: exiting due to model loading error" in s2:
                print("启动失败")
        
        #print(line.decode(), end='')
dic_status = {'status': False, 'msg': ''}

threading.Thread(target=local_model, daemon=True, args=(dic_status,)).start()

s = ""
while not dic_status['status']:
    if s != dic_status['msg']:
        s = dic_status['msg']
        print(s)
print("模型启动成功")


#!./llama-box -c 8192 -np 4 --host 0.0.0.0 -m /kaggle/input/my_model/gguf/glm-4-9b-chat-q4_k_s/1/glm-4-9b-chat-Q4_K_S.gguf
!curl http://localhost:8080/v1/completions -H "Content-Type: application/json" -d '{"model": "glm4", "stream" : false,"temperature" : 0.05, "top_p" : 0.85,"max_tokens" : 768, "frequency_penalty" : 0,"prompt": "<|system|>You are a helpful assistant.<|user|你好<|assistant|>"}'

In [ ]:
%cd /kaggle/working/KeywordGacha
with open("input.txt", "w", encoding='utf-8') as f:
    f.write("3\n1\n/kaggle/working/KeywordGacha/input\n")
!python /kaggle/working/KeywordGacha/main.py < input.txt


In [ ]:
!ls -l /kaggle/working/KeywordGacha/output

import shutil
shutil.make_archive('/kaggle/working/result', 'zip', '/kaggle/working/KeywordGacha/output')